# QISKIT Lab 6 - VQE H2 and LiH
 1. **S. G. Nana Engo**, serge.nana-engo@facsciences-uy1.cm
    * Department of Physics, Faculty of Science, University of Yaounde I
1. **J-P. Tchapet Njafa**, jean-pierre.tchapet-njafa@univ-maroua.cm
    * Department of Physics, Faculty of Science, University of Maroua
1. **P. Djorwe**, djorwepp@gmail.com
    * Department of Physics, Faculty of Science, University of Ngaoundere
       
March 2023

$$ 
\newcommand{\ad}{a^\dagger} % Operateur bosonique adjoint 
\newcommand{\T}{\mathtt{T}} %Operateur T quelconque
\newcommand{\U}{\mathtt{Z}} %Operateur Z quelconque
\newcommand{\HH}{\mathtt{U}} %Operateur U quelconque
\newcommand{\mt}[1]{\mathtt{#1}} %  Use mathtt
\newcommand{\mel}[3]{\langle #1|#2|#3\rangle} %Matrix element
\renewcommand{\ket}{|#1\rangle} 
$$

L'algorithme VQE (Variational Quantum Eigensolver), dans la contexte de la chimie quantique ou de la modélisation moléculaire, utilise la méthode variationelle, et deux processeurs, quantique et classique, pour déterminer l'énergie la plus basse associée à la valeur propre de l'etat fondamental ou des états excités.

## Algorithme détaillé de la VQE
On peut résumer cet algorithme en deux grandes parties qu'illustre la figure ci-dessous.

<center>
<img src="./Graphics/VQE_Diagram.png" width=400 />
</center>
                                                          

### Processeur quantique

Le processeur quantique comporte trois étapes fondamentales :

1. Définir le circuit quantique ou porte quantique $\mathtt{U}(\vec{\theta})$;
2. Préparer de la fonction d'essai paramétré $|\Psi (\vec{\theta})\rangle$ appelée **Ansatz**, qui est essentiellement une estimation de l'état fondamental, à cet effet, on choisit arbitrairement un état de référence $|\psi_0\rangle$ sur lequel on applique $\mathtt{U}(\vec{\theta})$,
	\begin{equation}
		|\Psi (\vec{\theta})\rangle= \mathtt{U}(\vec{\theta})|\psi_0 \rangle=\sum_i\alpha_i|E_i\rangle.
	\end{equation}
3. Mesurer de la valeur moyenne ou fonction de coût
\begin{equation}
C(\vec\theta)=\langle\Psi(\vec\theta)|\mathtt{H}|\Psi(\vec\theta)\rangle
=\langle \psi_0| \mathtt{U}^\dagger (\vec{\theta})\mathtt{HU} (\vec{\theta})|\psi_0\rangle.
\end{equation}
Selon la décomposition spectrale, $\mathtt{H}$ peut être représenté par:
\begin{equation}
\mathtt{H}=\sum_i E_i|E_i\rangle\langle E_i|.
\end{equation}
En vertu du [théorème variationnel](https://en.wikipedia.org/wiki/Vcorrespondariational_method_(quantum_mechanics)) de Rayleigh-Ritz, la valeur moyenne est toujours supérieure ou égale à la valeur propre $E_0$ la plus basse de l'Hamiltonien $\mathtt{H}$, qui correspond à l'état fondamental $|E_{\min}\rangle$:
\begin{equation}
C(\vec\theta)=\langle \psi_0| \mathtt{U}^\dagger (\vec{\theta})\mathtt{HU} (\vec{\theta})|\psi_0\rangle
=\sum_i|\alpha_i|^2E_i\geq E_{\min}.
\end{equation}
Le problème se résume à trouver un tel choix optimal de paramètres $\vec\theta=(\theta_1,\dots,\theta_n)^T$ à valeurs réelles, permettant de trouver la valeur moyenne minimale $E_{\min}$ qui est l'énergie de l'état fondamental et l'état correspondant est l'état fondamental $|E_{\min}\rangle$.


### Processeur classique

Grâce au processeur quantique, on obtient une valeur moyenne dépendante des paramètres. Cette valeur peut être minimisée avec une méthode d'optimisation qui permet d'ajuster les paramètres de l'état d'essai. L'algorithme procède alors de façon itérative, l'optimiseur classique proposant de nouvelles valeurs de paramètres pour l'état d'essai.

En gros, dans le processeur classique :
1. Minimiser la valeur moyenne ou fonction de coût $C(\vec\theta)$ en faisant varier les paramètres $\vec{\theta}$ de l'_Ansatz_, en utilisant un optimiseur classique.
2. Itèrer jusqu'à ce que le critère de convergence ($10^{-7}$) soit atteint et que $|\psi(\vec{\theta})\rangle\simeq |E_0(\vec{\theta})\rangle$.


### Défis ou challenges de la VQE

1. La taille de l'espace de Hilbert en fonction du nombre des opérations de portes nécessaires ou profondeur du circuit pour obtenir le précisions souhaitée pour les valeurs attendues (comme l'énergie du système).

2. Le nombre de portes parametrés (nombre de paramètres à optimiser) en fonction de l'amplitude des gradients pour variables de circuits.

3. La taille de l'espace de Hilbert en fonction du nombre de mesures nécessaire pour parvenir à la convergence des propriétés physiques (par exemple, l'énergie du système).


Dans ce qui suit, nous utilisons les **Unitary Coupled Clusters Singles and Doubles** (UCCSD) comme point de départ pour déterminer une fonction d'état d'essai pour la méthode variationnelle, car il est essentiel que l'ansatz VQE soit proche de l'état fondamental réel pour que les calculs VQE réussissent. Dans ce tutoriel, nous nous concentrons sur le calcul de l'état fondamental et de la surface d'énergie potentielle de Born-Oppenheimer (BOPES) pour les molécules d'hydrogène (H2) et d'hydrure de lithium (M) et une macromolécule.

### Unitary Coupled Cluster (UCC) *Ansatz*

En chimie quantique, la méthode **Unitary Coupled Cluster (UCC)** (de [cluster à couplage unitaire](https://fr.wikipedia.org/wiki/M%C3%A9thode_du_cluster_coupl%C3%A9)) apparait comme une extension de la méthode *Coupled Cluster (CC)* qui est une méthode de traitement de la corrélation électronique. Elle est basée sur l'expression de la fonction d'état à $N$ électrons comme une combinaison linéaire de déterminants de Slater incluant la fonction d'état HF de l'état fondamental et toutes les excitations possibles des orbitales occupées vers des orbitales inoccupées. Ainsi, il sera possible de générer un *Unitary Coupled Cluster Ansatz* en appliquant à un état de référence $\ket{\psi_0}$ un opérateur unitaire qui est une somme anti-Hermitienne d'opérateurs d'excitation et de désexcitation sous la forme $e^{\T(\theta)-\T^\dagger(\theta)}$, qui est un opérateur unitaire
$$
\ket{\psi(\theta)} = \mt{U}(\theta) \ket{\psi_0}
= e^{\T(\theta)-\T^\dagger(\theta)} \ket{\psi_0(\theta)} ,
$$
* $\theta$ est l'amplitude CC. Il représente aussi le paramètre d'optimisation pouvant prendre des valeurs réelles ou imaginaires. Mais les paramètres réels se sont révélés plus précises et plus réalisables ;

* $\T(\theta)$ est l'opérateur de *Cluster* ou opérateur d'excitation complète, défini comme  $\T(\theta)=\sum_{k=1}^N\T_k(\theta)$ avec $\T_k(\theta)$ l'opérateur d'excitation au $k$-ième ordre, qui contient des termes $k$-corps. Par exemple,
    * l'opérateur 
\begin{equation}
\T_1 = \underset{i\in\rm{unocc}}{\sum_{j\in\rm{occ}}}\theta_{ij}\ad_i a_j,
\end{equation}  
engendre les **excitations simples** $j\rightarrow i$ (transforme le déterminant HF de référence en une combinaison linéaire des déterminants monoexcités),
    * l'opérateur
 \begin{equation}
\T_2 = \underset{l>k\in\rm{occ}}{\sum_{i>j\in\rm{unocc}}}\theta_{ijkl} 
\ad_i\ad_j a_k a_l,\ \dots,
\end{equation}  
engendre les **doubles excitations** (transforme le déterminant HF de référence en une combinaison linéaire des déterminants doublement excités).
    * Les termes d'ordre supérieur (triple, quadruple, etc.) sont possibles, mais sont actuellement rarement pris en charge par les bibliothèques de chimie quantique.
    * "occ" et "unocc" sont définis comme les sites occupés et les sites inoccupés dans l'état de référence.
    * Les opérateurs $\ad_i$ et $a_i$ dans les termes de clusters couplés ci-dessus sont écrits dans une forme canonique, dans laquelle chaque terme est en ordre normal (opérateurs de création sont à gauche de tous les opérateurs d'annihilation).

### Workflow de la VQE 

<center><img src="Graphics/VQE_Flowchart.jpeg" width="500"/></center>


1. Renseigner la structure de la molécule.

2. Effectuer le calcul HF dans la base chimique indiquée. Il s'agit en réalité représenter le problème de l'equation de Schrödinguer électronique, $\mathtt{H}_{\rm el}|\Psi\rangle = E_{\rm el}|\Psi\rangle. $\mathtt{H}_{\rm el}$ est l'Hamiltonien de la classe `qiskit_nature.second_q.hamiltonians.ElectronicEnergy`

3. Extraire, à l'aide du calcul HF précédent, les éléments de matrice 1-integrals $h_{pq}$ et 2-integrals $h_{pqrs}$ qui relient l'Hamiltonien de la seconde quantification 1a celui de la première quantification. Les utiliser pour construire l'Hamiltonien fermionique de la seconde quantification
\begin{equation}
\mathtt{H} = h_0+\sum_{p,q=1}^M h_{pq}\ad_p a_q + \frac12\sum_{p,q,r,s=1}^M h_{pqrs}\ad_p \ad_q a_ra_s ,
\end{equation} 
que `qiskit_nature.second_q.mappers.QubitConverter` converti, grâce à un encodage approprié (JWT, PT ou BKT) en Hamiltonien qubit.

   * Exploiter les symmétries
   \begin{align}
   &[\mathtt{H},\mathtt{N_\uparrow}] = [\mathtt{H},\mathtt{N_\downarrow}] = 0, 
   && \mathtt{N_\uparrow} = \sum_{p=1}^{M/2} \alpha^\dagger_p \alpha_p,
   & \mathtt{N_\downarrow} = \sum_{p=M/2+1}^M \alpha^\dagger_p \alpha_p,
   \end{align}
   pour effectuer la **reduction 2-qubit** (une pour chaque symétrie $\mathbb{Z}$ de l'Hamiltionien) sans modifier la partie inférieure du spectre d'énergie (y compris l'état fondamental). $\mathtt{N_\downarrow}$ et $\mathtt{N_\uparrow}$ sont les opérateurs nombre de particules de spin down et up.

   * Utiliser `qiskit_nature.second_q.transformers.FreezeCoreTransformer` afin d'appliquer l'approximation du noyau gelé (**frozen-core approximation**) pour réduire le nombre possible des excitations uniques ou double et le nombre de qubits.

4. La fonction d'état d'essai ou *Ansatz* $\ket{\psi(\theta)}$ est généré à partir de l'état HF $\ket{\Phi_0}$ en appliquant les opérateurs de clusters q-UCC ou autre forme d'*Ansatz* variationnel (`qiskit_nature.second_q.circuit.library.UCCSD`).

5. Evaluer l'énergie du système $\mel{\psi(\theta)}{\mathtt{H}}{\psi(\theta)}$ en utilisant
   * la primitive `qiskit.primitives.Estimator`;
   * l'optimisation des paramètres du circuit (grâce à `qiskit.algorithms.optimizers`), suivi par la séquence de mesure des propriétés physiques. Ceci peut se faire avec ou sans les mesures du bruit.

6. Repeter les étapes (4) et (5) jusqu'à la convergence, en utilisant un optimiseur classique. Par usage, on fixe le critère de convergence à $10^{-7}$. En effet, il est essentiel que l'*Ansatz* VQE soit proche de l'état fondamental réel pour que les calculs VQE réussissent. Pour obtenir une estimation d'énergie précise de 1 milli-Hartree (mHA), l'*Ansatz* pour le VQE doit être proche du véritable état fondamental de moins d'un sur un million.

<center><img src="Graphics/Qiskit_nature_Flow.png" width="500"/></center>


## Molécule H2

Ici, nous calculons en fait uniquement la partie électronique. Lors de l'utilisation du package Qiskit Nature, l'énergie de répulsion nucléaire sera ajoutée automatiquement pour obtenir l'énergie totale de l'état fondamental.

### Structure électronique

Nous allons renseigner les données de la molécule et effectuer la conversion en créant un Hamiltonien fermionique (problème électronique précisement) qui sera ensuite converti par la transformation de Jordan Wigner, à un Hamiltonien qubit prêt pour le calcul quantique. 

-  On commence par définir la géométrie de la molécule d'hydrogène.

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver
from qiskit_nature.second_q.transformers import FreezeCoreTransformer

In [ ]:
H2_driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

# Electronic structure problem
H2_problem = H2_driver.run()
transformer = FreezeCoreTransformer()
H2_problem = transformer.transform(H2_problem)

- Configurons le mappeur et le convertisseur qubit (pour avoir la décomposition de Pauli de l'Hamiltonien). Pour des raisons de comparabilité, nous allons utiliser trois mappers (JWT, PT, BKT). A cet effet, définissons la fonction `qubit_converter()`.

In [ ]:
from qiskit_nature.second_q.mappers import (
    ParityMapper,
    JordanWignerMapper,
    BravyiKitaevMapper, 
    QubitConverter
)

def qubit_converter(mapper):
    """A mapper instance used to convert second quantized to qubit operators

    Args:
        mapper : Mapper string that can be "Parity" or "JordanWigner" or "Bravyi-Kitaev"
        
    Returns: The QubitConverter instance
    """    
    return QubitConverter(mapper, two_qubit_reduction=True)

### Solveur

Nous devons définir un solveur. Le solveur est l'algorithme par lequel l'état fondamental est calculé.

- Commençons par un exemple purement classique: le `qiskit.algorithms.minimum_eigensolvers.NumPyMinimumEigensolver`. Cet algorithme diagonise exactement l'Hamiltonien. Bien qu'il ne soit pas très performant, il peut être utilisé sur de petits systèmes pour vérifier les résultats des algorithmes quantiques.

In [ ]:
from qiskit.algorithms.minimum_eigensolvers import NumPyMinimumEigensolver

- Pour définir le solveur VQE, il faut quatre éléments essentiels :
  1. Une **forme variationnelle ou *ansatz***. Nous utilisons ici l'*ansatz* Unitary Coupled Cluster (UCC). La valeur par défaut est d'utiliser toutes les excitations simples et doubles. Cependant, le type d'excitation (S, D, SD) ainsi que d'autres paramètres peuvent être sélectionnés.
  2. Une primitive, qui sera ici `qiskit.primitives import Estimator` pour le calcul des valeurs moyennes;
  3. Un optimiseur classique;
  4. Un `factory` permettant une initialisation rapide d'un VQE avec l'*ansatz* UCC. Ici, nous utilisons le `qiskit_nature.second_q.algorithms.VQEUCCFactory` qui ajoutera la forme variationnelle `UCCSD` avec un état initial ou de référence `HartreeFock`, qui initialise l'occupation de nos qubits en fonction du problème que nous essayons de résoudre.

- Configurons l'*ansatz* et plus précisement, le circuit quantique paramétré, du VQE

In [ ]:
from qiskit_nature.second_q.circuit.library import UCCSD

ansatz = UCCSD()

- Configurons la primitive `Estimator` pour le VQE. Elle calcule les valeurs moyennes des circuits d'entrée et des propriétés physiques.

In [ ]:
from qiskit.primitives import Estimator

estimator = Estimator()

- Configurons l'optimisseur classique pour le VQE. Pour des raisons de comparabilité, nous allons configurer trois optimisseurs classiques.

In [ ]:
from qiskit.algorithms.optimizers import SPSA, SLSQP, L_BFGS_B

- Utilisons un `factory` pour assembler les composants de l'algorithme VQE. 

In [ ]:
from qiskit_nature.second_q.algorithms import VQEUCCFactory

### Calcul et résultats

 - Préparons le solveur d'état fondamental et exécutons-le pour calculer l'état fondamental de la molécule grâce à la classe `qiskit_nature.second_q.algorithms.GroundStateEigensolver`. Il s'agit d'envelopper notre `qiskit_nature.second_q.mappers.QubitConverter` et notre algorithme quantique `qiskit_nature.second_q.algorithms.VQEUCCFactory` (ou classique `qiskit_nature.second_q.algorithms.NumPyMinimumEigensolver`) dans un seul `GroundStateEigensolver`. Pour des raisons de commodité définissons la fonction `run_vqe()`.

Le workflow interne est le suivant :
1. générer les opérateurs de seconde quantification stockés dans notre `problem`" (electronic structure problem);
2. mapper (et potentiellement réduire) les opérateurs dans l'espace qubit;
3. finaliser la configuration de `VQEUCCFactory` en fonction du `problem`;
4. exécuter l'algorithme quantique sur l'opérateur Hamiltonien qubit;
5. une fois le critère de convergence vérifié, évaluer les proprités physiques supplémentaires de l'état fondamental déterminé.

In [ ]:
def run_vqe(name, problem, mapper, optimizer, solver, show=True):
    """Computing of the molecular ground state with the class `GroundStateEigensolver`

    Args:
        name (str):  a string of characters to be printed, such as 'NumPy exact solver'

        problem :  Electronic structure problem

        mapper : Mapper string that can be "Parity" or "JordanWigner" or "Bravyi-Kitaev"
        
        optimizer :  Optimizer string that can be either "SPSA" or "SLSQP" or "L_BFGS_B"

        solver :  Sting that can be either "NumPy" or "VQE"

        
    Returns:
          The ground state of the molecule

    """    
    # Mapper
    dmap = {"Jordan-Wigner": JordanWignerMapper(), "Parity": ParityMapper(),  "Bravyi-Kitaev": BravyiKitaevMapper()}
    for k, v in dmap.items():
        if k == mapper:        
            q_conv = qubit_converter(v)
    
    # Optimizer
    dopt = {"SPSA": SPSA(), "SLSQP": SLSQP(), "L_BFGS_B": L_BFGS_B()}
    for n, m in dopt.items():
        if n == optimizer:        
            vqe_factory = VQEUCCFactory(estimator=estimator, ansatz=ansatz, optimizer=m)
    
    # Factory
    dfact = {"VQE": vqe_factory,
            "NumPy": NumPyMinimumEigensolver()}
    for i, j in dfact.items():
        if i == solver:        
            factory = j
    
    # Ground state computation using a minimum eigensolver. Returns the solver.
    Algo = GroundStateEigensolver(q_conv, factory) 
    
    # Leveraging Qiskit Runtime
    start = time.time()
    # Compute Ground State properties.
    ground_state = Algo.solve(problem)
    elapsed = str(datetime.timedelta(seconds = time.time()- start))
    if show:
        print(f'Running the VQE using the {name} and {mapper} transformation with {optimizer} optimize')
        print(f'Elapsed time: {elapsed} \n')
        print(ground_state.total_energies[0])
    return name, mapper, optimizer, ground_state, elapsed


* Resulat VQE

In [ ]:
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
import time, datetime

res_vqe = run_vqe('VQE UCCSD Solver', H2_problem, mapper="Parity", optimizer="SPSA", solver="VQE")

- Résultat solveur exact NumPy.

In [ ]:
res_np = run_vqe('Numpy Exact Solver', H2_problem, mapper="Parity", optimizer="SPSA", solver="VQE")

* Nous calculons l'erreur relative entre les deux solvers avec le mapper "Parity".

In [ ]:
rel_error = (res_np[3].total_energies[0] - res_vqe[3].total_energies[0])*100/res_np[3].total_energies[0] 
print(f'\n The relative error between the two calculations : {rel_error:7.4f}%')

- Améliorons l'affichage des résultats

In [ ]:
print(f"Type of solver   | Mapper  | Optimizer | GS electronic energy| Rel. error |Elapsed time  | Optimizer time    ")
print('===================================================================================================================')
#print(f'Solver {name}   | {mapper}        |{optimizer}|{res_vqe.groundenergy}')
print(f'{res_vqe[0]}  | {res_vqe[1]}  |{res_vqe[2]}    |{res_vqe[3].total_energies[0]} |{rel_error:7.4f}%|{res_vqe[4]} | {res_vqe[3].raw_result.optimizer_time}')
#print('------------------------------------------------------------------------------------------')
print(f'{res_np[0]}| {res_np[1]}  |{res_np[2]}    |{res_np[3].total_energies[0]} |{rel_error:7.4f}%|{res_np[4]} | {res_np[3].raw_result.optimizer_time}')


On note que cette présentation n'est pas tout à fait satisfaisante.

### Visualisation de l'ensemble des résultats avec la bibliothèque `Pandas`

Afin d'analyser les résultats en fonction des `mappers`et des `optimizers` nous allons utiliser la bibliothèque `Pandas`.

A cet effet, il nous vaudra créer un repertoire dans lequel placé le fichier où sera imprimer les résultats au format `csv` que `pandas` va par la suite lire et afficher.

In [ ]:
# Creation of a simulation results folder
import os

cwd = os.getcwd()
directory = "Resultats"
targetPath = os.path.join(cwd, directory)

if not os.path.exists(targetPath):
    os.makedirs(targetPath)

In [ ]:
# File containing the results
QFile = os.path.join(targetPath, f"H2_results.csv")
q_result = open(QFile, "w")

q_result.write(f"Type of solver,Mapper,Optimizer,Ground state electronique energy,Relative error (%),Elapsed time,Optimizer time (s)\n")
q_result.close()

In [ ]:
Mapper = ["Parity","Jordan-Wigner","Bravyi-Kitaev"]
Optimizer = ["SPSA","SLSQP","L_BFGS_B"]

In [ ]:
q_result = open(QFile, "a")
for i in Mapper:
    for j in Optimizer:
        res_vqeH2 = run_vqe('VQE UCCSD Solver', H2_problem, mapper=i, optimizer=j, solver="VQE", show=False)
        res_npH2 = run_vqe('Numpy Exact Solver', H2_problem, mapper=i, optimizer=j, solver="VQE", show=False)
        rel_errorH2 = (res_npH2[3].groundenergy - res_vqeH2[3].groundenergy)*100/res_npH2[3].groundenergy
        q_result.write(f'{res_vqeH2[0]},{res_vqeH2[1]},{res_vqeH2[2]},{res_vqeH2[3].total_energies[0]},{rel_errorH2:7.4f},{res_vqeH2[4]},{res_vqeH2[3].raw_result.optimizer_time}\n')
        q_result.write(f'{res_npH2[0]},{res_npH2[1]},{res_npH2[2]},{res_npH2[3].total_energies[0]},{rel_errorH2:7.4f},{res_npH2[4]},{res_npH2[3].raw_result.optimizer_time}\n')
q_result.close()

In [ ]:
import pandas as pd

H2_data = pd.DataFrame(pd.read_csv(f"Resultats/H2_results.csv"))

H2_data

In [ ]:
# Grouping solver by type and mapper
H2_group = H2_data.sort_values(by=['Type of solver', 'Mapper'])

In [ ]:
# Grouping solver
H2_dataNP = H2_group[0:int(H2_data.shape[0]/2)]
H2_dataVQE = H2_group[int(H2_data.shape[0]/2):int(H2_data.shape[0])]

In [ ]:
# VQE UCCSD solver
H2_dataVQE

In [ ]:
# Numpy exact solver
H2_dataNP

## Plugin `qiskit_nature_solver`

Nous allons maintenant utiliser le plugin `qiskit_nature_pyscf` qui, nous le rappelons, couple PySCF et Qiskit Nature.  C'est un solveur [FCI](https://en.wikipedia.org/wiki/Full_configuration_interaction) (Full Configuration Interaction) basé sur Qiskit qui permet à un utilisateur de PySCF (Python-based Simulations of Chemistry Framework) de tirer parti des algorithmes quantique implémentés dans Qiskit pour être utilisés à la place de leurs homologues classiques (dans un esprit similaire à l'intégration NWChemEx).


In [ ]:
from pyscf import gto, scf, mcscf
from pyscf.mcscf import avas #AVAS method to construct mcscf active space

from qiskit_nature_pyscf import QiskitSolver

* Afin de ne pas interférer avec ce qui a été fait dans les sections précédentes, construisons à nouveau un algorithme quantique. Comme nous connaissons déja la procédure, nous n'allons plus la détailler.

In [ ]:
from qiskit_nature.second_q.mappers import (
    ParityMapper, 
    JordanWignerMapper,
    BravyiKitaevMapper,
    QubitConverter
)
from qiskit.primitives import Estimator
from qiskit_nature.second_q.algorithms import GroundStateEigensolver, VQEUCCFactory
from qiskit_nature.second_q.circuit.library import UCCSD
from qiskit.algorithms.optimizers import SLSQP

In [ ]:
def algorithm(mapper):
    mapper = mapper

    converter = QubitConverter(mapper, two_qubit_reduction=True)

    vqe = VQEUCCFactory(Estimator(), UCCSD(), SLSQP())

    algorithm = GroundStateEigensolver(converter, vqe)
    
    return algorithm

### Cas de la molécule d'hydrogène

In [ ]:
#  Initialisation de la structure moléculaire
H2_mol = gto.M(atom="H 0 0 0; H 0 0 .735", basis="sto-3g")

In [ ]:
# Calculs HF 
H2_h_f = scf.RHF(H2_mol).run()

# Calculs post-HF
norb, nelec, mo =avas.avas(H2_h_f,['H 1s','H 1s'])

H2_cas = mcscf.CASCI(H2_h_f, norb, nelec)

# Intégration de l'algorithme quantique
mapper = JordanWignerMapper()

H2_cas.fcisolver = QiskitSolver(algorithm(mapper))

H2_cas.run()

### Cas de la molécule d'hydride de lithium

<center><img src="Graphics/Lithium_hydride.png" width="150"/></center>



In [ ]:
LiH_mol = gto.M(atom="Li 0 0 0; H 0 0 1.6", basis="sto-3g")

LiH_h_f = scf.RHF(LiH_mol).run()

norb, nelec = 2, 2

LiH_cas = mcscf.CASCI(LiH_h_f, norb, nelec)

mapper = JordanWignerMapper()

LiH_cas.fcisolver = QiskitSolver(algorithm(mapper))

LiH_cas.run()

### Molecule d'eau

<center><img src="Graphics/Water_structure1.png" width="150" /></center>

In [ ]:
H2O_mol = gto.M(
  atom="O 0 0 0.115; H 0 0.754 -0.459; H 0 -0.754 -0.459",
  basis="sto6g",
)

H2O_h_f = scf.RHF(H2O_mol).run()

# norb, nelec, mo =avas.avas(H2O_h_f,['O 2p', 'H 1s','H 1s'])

H2O_cas = mcscf.CASSCF(H2O_h_f, 2, 2)

# H2O_cas = mcscf.CASSCF(H2O_h_f, norb, nelec)

mapper = BravyiKitaevMapper()

H2O_cas.fcisolver = QiskitSolver(algorithm(mapper))

H2O_cas.run()

# Macro molécule

Pour visualiser cette macromolécule (en relation avec le HIV), décommenter la cellule suivante si vous avez installé `ase` (atomic simulation environment).
> pip install ase -U

In [ ]:
# from ase import Atoms
# from ase.build import molecule
# from ase.visualize import view

# macro_ASE = Atoms('ONCHHHC', [(1.1280, 0.2091, 0.0000), 
#                           (-1.1878, 0.1791, 0.0000), 
#                           (0.0598, -0.3882, 0.0000),
#                           (-1.3085, 1.1864, 0.0001),
#                           (-2.0305, -0.3861, -0.0001),
#                           (-0.0014, -1.4883, -0.0001),
#                           (-0.1805, 1.3955, 0.0000)])

# view(macro_ASE, viewer='x3d')

In [ ]:
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo

M_atoms = ["O","N","C","H","H","H","C"]
M_coords = [(1.1280, 0.2091, 0.0000), 
                    (-1.1878, 0.1791, 0.0000),
                    (0.0598, -0.3882, 0.0000),
                    (-1.3085, 1.1864, 0.0001),
                    (-2.0305, -0.3861, -0.0001),
                    (-0.0014, -1.4883, -0.0001),
                    (-0.1805, 1.3955, 0.0000)]
M_charge = 0
M_multiplicity = 1
M_atom_pair=(6, 1)
M_info_dict={'atoms':M_atoms, 'coords':M_coords, 'charge':M_charge, 'multiplicity':M_multiplicity, 'atom_pair':M_atom_pair}

macromoleculeinfo = MoleculeInfo(M_atoms, M_coords, charge=M_charge, multiplicity=M_multiplicity)

In [ ]:
s = ''
k = 0
for atom in M_atoms:
  s += M_atoms[k] + ' ' + str(M_coords[k][0]) + ' ' + str(M_coords[k][1]) + ' ' + str(M_coords[k][2]) + '; '
  k += 1
s = s[0:-2]

M_mol = gto.M(atom = s)


In [ ]:
M_h_f = scf.RHF(M_mol).run()

norb, nelec = 2, 2

M_cas = mcscf.CASCI(M_h_f, norb, nelec)

mapper = BravyiKitaevMapper()

M_cas.fcisolver = QiskitSolver(algorithm(mapper))

M_cas.run()

In [ ]:
# Display Qiskit Software and System information
import qiskit.tools.jupyter
%qiskit_version_table